In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data, datasets
import numpy as np
import pandas as pd
import random
#use torchtext==0.3.1 only because stable and only for linux not for windows

In [ ]:
#dont change these values nor comment them
SEED = 1234 
torch.manual_seed(SEED) 
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy') 
LABEL = data.LabelField(dtype = torch.float) #datatype may be removed or change

In [ ]:
# when loading your own data and is in tabular for use these method otherwise dont run it WARNING

path = "root" # where your train and test data is present i.e root file
train_file = "train.csv" # name of the train file
test_file = "test.csv" #name of the test file
format = 'csv' #format of the files
fields=[('text', TEXT), ('labels', LABELS)] # in this 'text' is column in csv containing text, 'labels' in csv containing target TEXT, LABELS are preprocess

train_data, test_data = data.TabularDataset.splits(
    path=path, train=train_file ,
    test=test_file, format=format,
    fields=fields)

In [ ]:
#we are using IMDB dataset so we run this cell

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
#measure the size of our dataset

print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [ ]:
# lets see out first examples of train dataset
print(vars(train_data.examples[0]))

{'text': ['Bela', 'made', '9', 'pics', 'for', 'Monogram', ',', 'but', 'it', 'was', 'only', 'at', 'THIS', 'one', ',', 'the', '4TH', ',', 'that', 'things', 'started', 'to', 'come', 'together', '.', 'All', 'the', 'rest', 'in', 'the', 'series', 'would', 'use', 'this', 'one', 'as', 'the', 'essential', 'template', 'for', 'production', ',', 'writing', 'and', 'character', 'development', '.', 'From', 'here', 'on', ',', 'better', 'or', 'worse', ',', 'the', 'series', 'would', 'also', 'deal', 'with', 'one', 'essential', 'theme', ':', 'a', 'scientist', '(', 'usually', 'Bela', ')', 'makes', 'experiments', 'in', 'the', 'basement', 'or', 'the', 'old', 'house', '(', 'sometimes', 'IN', 'the', 'basement', 'in', 'the', 'old', 'house', ')', 'that', 'causes', 'things', 'to', 'go', 'blooey', '.', 'This', 'was', 'also', 'the', 'first', 'time', 'that', 'Art', 'Director', 'Dave', 'Milton', 'got', 'a', 'chance', 'to', 'spread', 'his', 'wings', '.', 'He', 'came', 'on', 'board', 'for', 'BLACK', 'DRAGONS', ',', 'th

In [ ]:
#train valid split

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [ ]:
MAX_VOCAB_SIZE = 100000

# build the vocabulary on our datas
#TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)


# use this one if want to use better embedding method it uses glove
TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.200d", 
                 unk_init = torch.Tensor.normal_)


LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 100002
Unique tokens in LABEL vocabulary: 2


In [ ]:
# print all the labels
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f28814eb0d0>, {'neg': 0, 'pos': 1})


In [ ]:
# print 20 most common text vocabulary
print(TEXT.vocab.freqs.most_common(20))

[('the', 203136), (',', 193205), ('.', 166263), ('a', 109768), ('and', 109620), ('of', 100942), ('to', 93838), ('is', 76509), ('in', 61486), ('I', 54356), ('it', 53797), ('that', 49631), ('"', 44781), ("'s", 43931), ('this', 42570), ('-', 36991), ('/><br', 35552), ('was', 35000), ('as', 30632), ('with', 30157)]


In [ ]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [ ]:
# create the model 
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]

        # torch.permute is use to swap the axis
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = 1 # for just binary classification
# OUTPUT_DIM = len(LABEL.vocab) # for multiclass classification uncomment this
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
print('INPUT_DIM:',INPUT_DIM,"\nEMBEDDING_DIM:",EMBEDDING_DIM,"\nN_FILTERS:",N_FILTERS,"\nFILTER_SIZES:",FILTER_SIZES,"\nOUTPUT_DIM:",OUTPUT_DIM,"\nPAD_IDX:",PAD_IDX)

INPUT_DIM: 100002 
EMBEDDING_DIM: 200 
N_FILTERS: 100 
FILTER_SIZES: [2, 3, 4] 
OUTPUT_DIM: 1 
PAD_IDX: 1


In [ ]:
print(TEXT.vocab.stoi[TEXT.pad_token])

1


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,181,001 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings)
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.8542,  0.4022,  0.4238],
        [ 0.2078,  1.1879, -0.7320,  ...,  1.3663, -0.4598,  0.6668],
        [-0.0715,  0.0935,  0.0237,  ...,  0.3362,  0.0306,  0.2558],
        ...,
        [ 0.9199,  1.6288, -0.7184,  ..., -0.3771,  0.3570, -0.6618],
        [ 0.7744,  0.6477,  0.0234,  ..., -0.1684, -0.6374,  0.3421],
        [-0.1095, -1.0616, -0.7907,  ..., -1.0804, -0.2631, -1.3688]])


tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.8542,  0.4022,  0.4238],
        [ 0.2078,  1.1879, -0.7320,  ...,  1.3663, -0.4598,  0.6668],
        [-0.0715,  0.0935,  0.0237,  ...,  0.3362,  0.0306,  0.2558],
        ...,
        [ 0.9199,  1.6288, -0.7184,  ..., -0.3771,  0.3570, -0.6618],
        [ 0.7744,  0.6477,  0.0234,  ..., -0.1684, -0.6374,  0.3421],
        [-0.1095, -1.0616, -0.7907,  ..., -1.0804, -0.2631, -1.3688]])

In [ ]:
#Then zero the initial weights of the unknown and padding tokens.
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [ ]:
#weight_decay=0.01
optimizer = optim.Adam(model.parameters(),lr=0.001)

#criterion = nn.CrossEntropyLoss() #for multi-class uncomment this
criterion = nn.BCEWithLogitsLoss() # for two class

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
# use for two class classification
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
# use for multiple class classification
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [ ]:
def train(model, iterator, optimizer, criterion):    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label) # for binary class
        #acc = categorical_accuracy(predictions, batch.label) # for multiclass uncomment this
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label) # for two class
            #acc = categorical_accuracy(predictions, batch.label) # for mlticlass uncomment this

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model2.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 29s
	Train Loss: 0.019 | Train Acc: 99.50%
	 Val. Loss: 0.480 |  Val. Acc: 86.25%


In [ ]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.450 | Test Acc: 84.40%


In [ ]:

import spacy
nlp = spacy.load('en')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = torch.sigmoid(model(tensor))
    #max_preds = preds.argmax(dim = 1) # for multiclass
    return preds[0][0]#round(float(preds[0][0]))

In [ ]:
predict_class(model, "i hate it")

tensor(0.4811, device='cuda:0', grad_fn=<SelectBackward>)

In [ ]:
pred_class = round(float(predict_class(model, "The movie is quite a funny one you take birth and you die only to realize it was a character assessment test and your girlfriend leaves you because you have been a horrible person.")))
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = neg


In [ ]:
def save_vocab(vocab, path):
    import pickle
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()

In [ ]:
torch.save(TEXT.vocab,'vocab.pt')

In [ ]:
torch.save(LABEL.vocab, 'label.pt')

In [ ]:
objects = torch.load("vocab.pt")

In [ ]:
import torchtext
torchtext.__version__

'0.3.1'